In [2]:
from rur import uri, uhmi
from rur.sci import photometry

import numpy as np
from tqdm import tqdm

import time
from tree_utool import *

In [3]:
modes = ['y07206', 'y05420']
mode = modes[1]

repo, rurmode, dp = mode2repo(mode)
nout = load_nout(mode=mode, galaxy=True)
nstep = load_nstep(mode=mode, galaxy=True, nout=nout)
iout = nout[0]
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
print(len(gals))

path = f"/home/jeon/YoungTree/result/{mode}"
file_list = os.listdir(path)
file_list = [file for file in file_list if (file.startswith(f"Progenitor_Branch") & file.endswith(".pickle"))]
print(len(file_list))

Age of the universe (now/z=0): 13.741 / 13.741 Gyr, z = -0.00000 (a = 1.0000)
1289
1289


In [5]:
assert len(gals)==len(file_list)

In [6]:
dtype = [("rootid", int), ("rootiout", int), ("id", int), ("timestep", int), ("score", float), ("elapsed", float)]
first = True
for gal in gals:
    readme, root, elap, branch, scores = pklload(f'/home/jeon/YoungTree/result/{mode}/Progenitor_Branch_ID{gal["id"]:05d}_iout{iout:05d}.pickle')

    outs = list( branch.keys() )
    ntree = len(outs)
    rootids = np.full(ntree, root['id'], dtype=int)
    rootiouts = np.full(ntree, root['timestep'], dtype=int)
    elapseds = np.full(ntree, elap, dtype=float)
    
    ids = np.array( [branch[ith]['id'] for ith in outs] )
    timesteps = np.array( [branch[ith]['timestep'] for ith in outs] )
    scores = np.array( [scores[ith] for ith in outs] )
    if first:
        result = np.rec.fromarrays((rootids, rootiouts, ids, timesteps, scores, elapseds), dtype=dtype)
        first = False
    else:
        result = np.hstack((result, np.rec.fromarrays((rootids, rootiouts, ids, timesteps, scores, elapseds), dtype=dtype)))
print(len(result))

186241


In [8]:
if not os.path.isdir(f"{repo}/YoungTree"):
    os.mkdir(f"{repo}/YoungTree")
readme = "README:\n    1) Root galaxy ID\n    2) Root galaxy iout\n    3) Galaxy ID\n    4) Galaxy iout\n    5) Score\n    6) Elapsed computing time"
pklsave((readme, result), f"{repo}/YoungTree/ytree.pickle")